# 1. Импорты

In [ ]:
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC

from sklearn.metrics import roc_auc_score, roc_curve